In [109]:
import math
cost_dict = {'w': math.inf, 'm': 100, 'p': 10, 's': 30}
mat = None
path = ""

class Node():
#Captures parent and current location of node

    def __init__(self, parent=None, location=None):
        self.parent = parent
        self.location = location
        

def get_tile(observable, direction):
    return mat[observable[direction][0]][observable[direction][1]]


def surrounding_spaces(curr, maxlen):
    # Check if the agent is near the border of the maze
    x = curr.location[0]
    y = curr.location[1]
    observable = {}
    # North Neighbor
    if x-1 >= 0:
        observable['n'] = (x-1, y)
    # East Neighbor
    if y+1 <= maxlen - 1:
        observable['e'] = (x, y+1)
    # South Neighbor
    if x+1 <= maxlen - 1:
        observable['s'] = (x+1,y)
    # West Neighbor
    if y-1 >= 0:
        observable['w'] = (x, y-1)
    # North east neighbor
    if x-1 >= 0 and y+1 <= maxlen - 1 and not (get_tile(observable, 'n') == get_tile(observable, 'e') == 'w'):
        observable['ne'] = (x-1, y+1)
    # South East Neighbor
    if x+1 <= maxlen - 1 and y+1 <= maxlen - 1 and not (get_tile(observable, 's') == get_tile(observable, 'e') == 'w'):
        observable['se'] = (x+1, y+1)
    # South West Neighbor
    if x+1 <= maxlen - 1 and y-1 >= 0 and not (get_tile(observable, 's') == get_tile(observable, 'w') == 'w'):
        observable['sw'] = (x+1, y-1)
    # North West Neighbor
    if x-1 >= 0 and y-1 >= 0 and not (get_tile(observable, 'n') == get_tile(observable, 'w') == 'w'):
        observable['nw'] = (x-1, y-1)

    return observable


def find_movable(observable):
    # returns possible moves for agent.
    movable = {}
    for i in observable.keys():
        if i in ['n','w','s','e'] and mat[observable[i][0]][observable[i][1]] != 'w':
            x = observable[i][0]
            y = observable[i][1]
            movable[i] = observable[i]
    return movable



# Building heuristic based on number_times_visited for a tile and locations(sand,mountainn and path)
def find_next_move(curr, observable, explored):
    # returns best next move
    global path
    movable_dict = find_movable(observable)
    #movable_dict : map <direction,location tuple>
    #explored : map<location, no. of times visited>
    scores={} # location tuple vs cost
    for k,v in movable_dict.items():
        #print(cost_dict[mat[v[0]][v[1]]],"path cost")
        #print(explored[v],"times visited")
        scores[k]=100*explored[v] + cost_dict[mat[v[0]][v[1]]]
    #list_of_movable = sorted(movable_dict.keys(), key=lambda x: explored[movable_dict[x]])#priority to unexplored or less visited
    #path += list_of_movable[0]
    dir = min(scores.items(), key=lambda x: x[1])[0]
    #print(scores,"score")
    #print(dir,"dir")
    path += dir
    return Node(curr, observable[dir])#observable[list_of_movable[0]])


def populate_explored(explored, observable):
    for v in observable.values():
        if v not in explored:
            explored[v] = 0


def update_explored(curr, explored):
    #updates explored location states. i.e. increments location pointer according to number of visits.
    explored[curr.location] += 1


def solve(start, goal, matrix):
    global mat
    mat = matrix
    start_node = Node(None, start)
    curr = start_node
    observable = surrounding_spaces(curr, len(mat))
    explored = dict()  # keeping track of what is explored
    explored[start_node.location] = 0
    steps = 0
    while goal not in observable.values():
        populate_explored(explored, observable)
        update_explored(curr, explored)
        curr = find_next_move(curr, observable, explored)
        steps += 1
        #print(show_exploredmap(len(mat), explored))
        observable = surrounding_spaces(curr, len(mat))
    reachGoalInObservable(goal, observable)
    observable
    return path.upper()

def reachGoalInObservable(goal, observable):
    movable_dict = find_movable(observable)
    direction= ""
    global path
    print(observable)
    for k,v in observable.items():
            if v == goal:
                direction = k
                break
                
    if goal in movable_dict.values():
        #in N,S,E,W, i.e. non diagnal
        path+=direction
        
    else:
        #see diagnals only
        f = direction[0]
        s = direction[1]
        fTile = get_tile(observable,f)
        sTile = get_tile(observable,s)
        fCost = cost_dict[fTile]
        sCost = cost_dict[sTile]
        if fCost>sCost:
            path += s+f
        else:
            path += f+s
            
def show_exploredmap(length, explored):
    matrix =[[0 for _ in range(length)] for _ in range(length)]
    for k in explored.keys():
        matrix[k[0]][k[1]] = explored[k]
    for i in range(0,length):
        print(matrix[i])



if __name__ == '__main__':
    mat = [['w', 'w', 'w', 'w', 'w', 'w', 'w', 'p'],
           ['m', 'm', 's', 's', 's', 'p', 'p', 'w'],
           ['s', 's', 'p', 's', 'p', 's', 'm', 'w'],
           ['p', 'w', 'm', 's', 's', 's', 'w', 'w'],
           ['w', 'w', 'm', 'w', 'w', 'm', 'p', 'p'],
           ['w', 'p', 'w', 'w', 's', 'p', 'w', 'm'],
           ['m', 's', 's', 'w', 'w', 'p', 'p', 'p'],
           ['s', 's', 'p', 'm', 'm', 'p', 'w', 'w']]



    #path = solve((0, 0), (7,0), world)
    path = solve((6, 7), (6,2), mat)
    
    print(path)


{'n': (6, 3), 'e': (7, 4), 'w': (7, 2), 'ne': (6, 4), 'nw': (6, 2)}
WWNWENEESNWWNNNESNWWSSWNWNEESESWWWSNNWWSNNEEESENEESWSWWNWWNWSSNSNENEEEEESWWSESEESSWWSWWWN


In [63]:
import random
 
def generate_world(limit):
     dimension = limit#random.randint(2, limit)
     world = [[None]*dimension for i in range(dimension)]

     goal = (random.randint(0, dimension-1),random.randint(0, dimension-1))
     start = (random.randint(0, dimension-1),random.randint(0, dimension-1))

     types = ['p','s','m','w']
     for i in range(dimension):
         for j in range(dimension):
             world[i][j] = random.choice(types)

     return world


In [58]:
world=generate_world(50)

In [59]:
world

[['w',
  'w',
  'w',
  'm',
  'w',
  'p',
  's',
  'p',
  'w',
  's',
  'p',
  's',
  'm',
  's',
  's',
  'w',
  'm',
  'p',
  's',
  's',
  'p',
  's',
  'm',
  's',
  's',
  'p',
  's',
  's',
  's',
  'p',
  'm',
  's',
  'm',
  'm',
  's',
  's',
  's',
  's',
  'p',
  's',
  'm',
  'm',
  'p',
  'p',
  'm',
  'w',
  'm',
  's',
  's',
  's'],
 ['p',
  's',
  's',
  'p',
  's',
  'm',
  's',
  'p',
  'm',
  'p',
  'p',
  'p',
  'p',
  's',
  'w',
  's',
  's',
  's',
  'm',
  'p',
  'w',
  's',
  's',
  'm',
  'w',
  'm',
  'w',
  'm',
  'w',
  'w',
  's',
  'p',
  'm',
  'p',
  'p',
  'w',
  'w',
  's',
  's',
  's',
  'm',
  'w',
  'm',
  's',
  'p',
  's',
  'p',
  'm',
  's',
  'm'],
 ['s',
  'm',
  'p',
  's',
  'p',
  'w',
  'm',
  'w',
  'm',
  'w',
  'w',
  'm',
  's',
  'w',
  'p',
  'p',
  'w',
  'm',
  'p',
  'm',
  'm',
  'p',
  's',
  'p',
  'p',
  's',
  's',
  's',
  'p',
  'm',
  'p',
  'm',
  'w',
  's',
  'm',
  'p',
  'm',
  'm',
  'p',
  's',
  'w',
  'w',
  'w